#### P.S.  
Загрузка и обоснование датасета и метрик в [lab1.ipynb](lab1.ipynb)

# Логистическая и линейная регрессия

--------
## Задача классификации

In [81]:
# импортируем либы
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

#### Загрузим датасет 

In [82]:
path = 'winequality-red.csv'
data_class = pd.read_csv(path)

print("Первые 5 строк датасета для классификации:")
print(data_class.head())
print(f"\nРазмер датасета: {data_class.shape}")


Первые 5 строк датасета для классификации:
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  

#### (по аналогии с 1 лабой) создаем целевую переменную для классификации

In [83]:
data_class['wine_quality'] = data_class['quality'].apply(lambda x: 'good' if x >= 6 else 'bad')
data_class = data_class.drop('quality', axis=1)

print("\nРаспределение качества вина:")
print(data_class['wine_quality'].value_counts())


Распределение качества вина:
wine_quality
good    855
bad     744
Name: count, dtype: int64


#### Разделим датасет на признаки и целевую переменную

In [84]:
X = data_class.drop('wine_quality', axis=1)
y = data_class['wine_quality']

#### Масштабируем признаки

In [85]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
print(f"Размер обучающей выборки: {X_train.shape}, Размер тестовой выборки: {X_test.shape}")

Размер обучающей выборки: (1279, 11), Размер тестовой выборки: (320, 11)


#### Применим встроенный алгоритм

In [86]:
lr_baseline = LogisticRegression(max_iter=1000, random_state=42)
lr_baseline.fit(X_train, y_train)

y_pred = lr_baseline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Бейзлайн (Logistic Regression):")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Бейзлайн (Logistic Regression):
Accuracy: 0.7406
F1-Score: 0.7409

Classification Report:
              precision    recall  f1-score   support

         bad       0.71      0.74      0.73       149
        good       0.77      0.74      0.75       171

    accuracy                           0.74       320
   macro avg       0.74      0.74      0.74       320
weighted avg       0.74      0.74      0.74       320



#### Подберем гиперпараметры для оптимизации бейзлайна

##### `C: [0.01, 0.1, 1, 10, 100]`
- **Параметр регуляризации** - контролирует силу регуляризации
- **Малые значения (0.01-0.1)**: сильная регуляризация, предотвращает переобучение
- **Средние значения (1)**: баланс между fitting и регуляризацией (значение по умолчанию)
- **Большие значения (10-100)**: слабая регуляризация, модель ближе к обычной MLE
- **Логарифмическая шкала**: эффективный поиск по порядкам величин

##### `solver: ['liblinear', 'lbfgs', 'sag']`
- **liblinear**: эффективен для небольших датасетов, поддерживает L1 и L2
- **lbfgs**: стабильный алгоритм, хорош для небольших данных, только L2
- **sag**: стохастический метод, эффективен для больших данных, только L2
- Покрытие разных сценариев размера данных и вычислительных потребностей

##### `penalty: ['l2']`
- **L2 регуляризация (Ridge)**: сплошное сглаживание коэффициентов
- **Не включает L1**: так как не все солверы поддерживают L1 (lbfgs, sag - только L2)
- **Фокус на стабильность**: L2 обычно более стабилен и менее склонен к переобучению

##### Общая стратегия
Баланс между силой регуляризации (C) и выбором эффективного алгоритма оптимизации (solver) для разных характеристик данных.

In [87]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs', 'sag'],
    'penalty': ['l2']
}

grid_search = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42), param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Лучшие параметры: {best_params}")

Fitting 5 folds for each of 15 candidates, totalling 75 fits
Лучшие параметры: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}


#### Обучение с подобранными гиперпараметрами

In [88]:
best_lr = grid_search.best_estimator_

y_pred_best = best_lr.predict(X_test)

accuracy_best = accuracy_score(y_test, y_pred_best)
f1_best = f1_score(y_test, y_pred_best, average='weighted')

print("Улучшенный бейзлайн:")
print(f"Accuracy: {accuracy_best:.4f}")
print(f"F1-Score: {f1_best:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best))

Улучшенный бейзлайн:
Accuracy: 0.7375
F1-Score: 0.7378

Classification Report:
              precision    recall  f1-score   support

         bad       0.71      0.74      0.73       149
        good       0.77      0.73      0.75       171

    accuracy                           0.74       320
   macro avg       0.74      0.74      0.74       320
weighted avg       0.74      0.74      0.74       320



#### Имплементация алгоритма

In [89]:
class CustomLogisticRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        y_encoded = self._one_hot_encode(y)
        self.weights = np.zeros((X.shape[1], len(self.classes_)))
        self.bias = np.zeros(len(self.classes_))

        for _ in range(self.n_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            probabilities = self._softmax(linear_model)
            dw = np.dot(X.T, (probabilities - y_encoded)) / X.shape[0]
            db = np.sum(probabilities - y_encoded, axis=0) / X.shape[0]

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        probabilities = self._softmax(linear_model)
        predictions = np.argmax(probabilities, axis=1)
        return self.classes_[predictions]

    def _softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def _one_hot_encode(self, y):
        y_encoded = np.zeros((y.shape[0], len(self.classes_)))
        for idx, class_ in enumerate(self.classes_):
            y_encoded[:, idx] = (y == class_).astype(float)
        return y_encoded

custom_lr = CustomLogisticRegression()
custom_lr.fit(X_train, y_train)

y_pred_custom = custom_lr.predict(X_test)

accuracy_custom = accuracy_score(y_test, y_pred_custom)
f1_custom = f1_score(y_test, y_pred_custom, average='weighted')

print("Имплементация Custom Logistic Regression:")
print(f"Accuracy: {accuracy_custom:.4f}")
print(f"F1-Score: {f1_custom:.4f}")

custom_lr_improved = CustomLogisticRegression(
    learning_rate=0.1, 
    n_iterations=1000
)
custom_lr_improved.fit(X_train, y_train)
y_pred_custom_improved = custom_lr_improved.predict(X_test)

accuracy_custom_improved = accuracy_score(y_test, y_pred_custom_improved)
f1_custom_improved = f1_score(y_test, y_pred_custom_improved, average='weighted')

print("Custom Logistic Regression с улучшенными параметрами:")
print(f"Accuracy: {accuracy_custom_improved:.4f}")
print(f"F1-Score: {f1_custom_improved:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_custom))

Имплементация Custom Logistic Regression:
Accuracy: 0.7375
F1-Score: 0.7378
Custom Logistic Regression с улучшенными параметрами:
Accuracy: 0.7406
F1-Score: 0.7409

Classification Report:
              precision    recall  f1-score   support

         bad       0.70      0.75      0.73       149
        good       0.77      0.73      0.75       171

    accuracy                           0.74       320
   macro avg       0.74      0.74      0.74       320
weighted avg       0.74      0.74      0.74       320



#### Сравнение полученных результатов

In [90]:
print("Сравнение результатов для задачи классификации:")
print(f"Бейзлайн Accuracy: {accuracy:.4f}, Улучшенный Accuracy: {accuracy_best:.4f}, Custom Accuracy: {accuracy_custom:.4f}")
print(f"Бейзлайн F1-Score: {f1:.4f}, Улучшенный F1-Score: {f1_best:.4f}, Custom F1-Score: {f1_custom:.4f}")

Сравнение результатов для задачи классификации:
Бейзлайн Accuracy: 0.7406, Улучшенный Accuracy: 0.7375, Custom Accuracy: 0.7375
Бейзлайн F1-Score: 0.7409, Улучшенный F1-Score: 0.7378, Custom F1-Score: 0.7378


------

## Задача регрессии

In [91]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# загружаем данные
path = 'hour.csv'
data_reg = pd.read_csv(path)

print("Первые 5 строк датасета для регрессии:")
print(data_reg.head())
print(f"\nРазмер датасета: {data_reg.shape}")

Первые 5 строк датасета для регрессии:
   instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1   0        0        6           0   
1        2  2011-01-01       1   0     1   1        0        6           0   
2        3  2011-01-01       1   0     1   2        0        6           0   
3        4  2011-01-01       1   0     1   3        0        6           0   
4        5  2011-01-01       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1  

Размер датасета: (17379, 17)


####  Разделим данные на тестовую и обучающую выборку и масштабирование признаков 

In [92]:
# убрал дату в строковом представлении (есть дата в числовых колонках), так же число, которое предсказываем (cnt)
useful_columns = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 
                  'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'registered']

# Разделим данные на тестовую и обучающую выборку
X = data_reg[useful_columns]
y = data_reg['cnt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nРазмер обучающей выборки: {X_train.shape}, Размер тестовой выборки: {X_test.shape}")
print(f"Используемые признаки: {list(X.columns)}")

scaler_reg = StandardScaler()
X_train_scaled = scaler_reg.fit_transform(X_train)
X_test_scaled = scaler_reg.transform(X_test)


Размер обучающей выборки: (13903, 13), Размер тестовой выборки: (3476, 13)
Используемые признаки: ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'registered']


#### Обучение бейзлайна и оценка модели 

In [93]:
lr_baseline = LinearRegression()
lr_baseline.fit(X_train_scaled, y_train)
y_pred_reg = lr_baseline.predict(X_test_scaled)

def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, r2

print("Бейзлайн (Linear Regression):")
mae_baseline, mse_baseline, r2_baseline = evaluate_model(y_test, y_pred_reg)
print(f"MAE: {mae_baseline:.4f}, MSE: {mse_baseline:.4f}, R^2: {r2_baseline:.4f}")

Бейзлайн (Linear Regression):
MAE: 22.0337, MSE: 1057.7405, R^2: 0.9666


#### Подбор гиперпараметров для оптимизации бейзлайна 

##### `alpha: [0.01, 0.1, 1, 10, 100]`
- **Параметр регуляризации** - контролирует силу L2-регуляризации
- **Малые значения (0.01-0.1)**: слабая регуляризация, модель близка к обычной линейной регрессии
- **Средние значения (1)**: умеренная регуляризация, баланс между bias и variance
- **Большие значения (10-100)**: сильная регуляризация, коэффициенты сильно сжимаются к нулю
- **Логарифмическая шкала**: эффективный поиск по разным порядкам величины регуляризации  

Берем только alpha тк это ключевой параметр, определяющий силу регуляризации. Для Ridge регрессии часто достаточно подобрать только alpha, а так же помогает уменьшению пространства поиска ускоряет GridSearch 

### Общая стратегия
Систематический поиск оптимальной силы регуляризации от очень слабой до очень сильной через логарифмическую шкалу значений.

In [94]:
param_grid = {"alpha": [0.01, 0.1, 1, 10, 100]}

grid_ridge = GridSearchCV(Ridge(), param_grid, cv=5, scoring="r2", verbose=1)
grid_ridge.fit(X_train_scaled, y_train)

best_alpha = grid_ridge.best_params_["alpha"]
print(f"Лучший alpha для Ridge регрессии: {best_alpha}")

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Лучший alpha для Ridge регрессии: 10


#### Обучение с подобранными гиперпараметрами 

In [95]:
ridge_optimized = Ridge(alpha=best_alpha)
ridge_optimized.fit(X_train_scaled, y_train)
y_pred_ridge = ridge_optimized.predict(X_test_scaled)

print("Улучшенная модель (Ridge Regression):")
mae_ridge, mse_ridge, r2_ridge = evaluate_model(y_test, y_pred_ridge)
print(f"MAE: {mae_ridge:.4f}, MSE: {mse_ridge:.4f}, R^2: {r2_ridge:.4f}")

Улучшенная модель (Ridge Regression):
MAE: 22.0182, MSE: 1057.5906, R^2: 0.9666


#### Имплементация алгоритма

In [96]:
class CustomRidgeRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000, alpha=1.0):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.alpha = alpha  # параметр регуляризации

    def fit(self, X, y):
        self.m = X.shape[0]
        self.n = X.shape[1]
        self.weights = np.zeros(self.n)
        self.bias = 0

        for _ in range(self.n_iterations):
            y_predicted = self._predict(X)
            # Добавляем регуляризацию L2
            dw = -(2/self.m) * np.dot(X.T, (y - y_predicted)) + (2*self.alpha/self.m) * self.weights
            db = -(2/self.m) * np.sum(y - y_predicted)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        return self._predict(X)

    def _predict(self, X):
        return np.dot(X, self.weights) + self.bias


custom_lr = CustomRidgeRegression()
custom_lr.fit(X_train_scaled, y_train)
y_pred_custom = custom_lr.predict(X_test_scaled)

mae_custom, mse_custom, r2_custom = evaluate_model(y_test, y_pred_custom)

print("Custom Linear Regression:")
print(f"MAE: {mae_custom:.4f}, MSE: {mse_custom:.4f}, R^2: {r2_custom:.4f}")

custom_ridge = CustomRidgeRegression(learning_rate=0.01, n_iterations=1000, alpha=best_alpha)
custom_ridge.fit(X_train_scaled, y_train)
y_pred_custom_ridge = custom_ridge.predict(X_test_scaled)

mae_custom_ridge, mse_custom_ridge, r2_custom_ridge = evaluate_model(y_test, y_pred_custom_ridge)

print("Улучшенный Custom Ridge Regression:")
print(f"MAE: {mae_custom_ridge:.4f}, MSE: {mse_custom_ridge:.4f}, R^2: {r2_custom_ridge:.4f}")

Custom Linear Regression:
MAE: 22.0337, MSE: 1057.6337, R^2: 0.9666
Улучшенный Custom Ridge Regression:
MAE: 22.0197, MSE: 1057.5042, R^2: 0.9666


#### Сравнение всех полученных результатов 

In [97]:
print("\nСравнение результатов для задачи регрессии:")
print(f"Бейзлайн MAE: {mae_baseline:.4f}, Улучшенный MAE: {mae_ridge:.4f}, Custom MAE: {mae_custom:.4f}, Улучшенный Custom MAE: {mae_custom_ridge:.4f}")
print(f"Бейзлайн MSE: {mse_baseline:.4f}, Улучшенный MSE: {mse_ridge:.4f}, Custom MSE: {mse_custom:.4f}, Улучшенный Custom  MSE: {mse_custom_ridge:.4f}")
print(f"Бейзлайн R^2: {r2_baseline:.4f}, Улучшенный R^2: {r2_ridge:.4f}, Custom R^2: {r2_custom:.4f}, Улучшенный Custom R^2: {r2_custom_ridge:.4f}")


Сравнение результатов для задачи регрессии:
Бейзлайн MAE: 22.0337, Улучшенный MAE: 22.0182, Custom MAE: 22.0337, Улучшенный Custom MAE: 22.0197
Бейзлайн MSE: 1057.7405, Улучшенный MSE: 1057.5906, Custom MSE: 1057.6337, Улучшенный Custom  MSE: 1057.5042
Бейзлайн R^2: 0.9666, Улучшенный R^2: 0.9666, Custom R^2: 0.9666, Улучшенный Custom R^2: 0.9666
